# Google Cloud Platform Project Creation Workbook 
 
Use this workbook to create a google cloud project with everything needed to collect new data and host your own web app. 
 
Prerequisites:  
+ Create Google user account  <br><br>
+ Create your own personal Google Cloud Project and Enable Billing
    - Enable Free Tier account by seleting "Try it Free" here: [Try Google Cloud Platform for free](https://cloud.google.com/cloud-console)
    - Follow steps to activate billing found here: [Create New Billing Account](https://cloud.google.com/billing/docs/how-to/manage-billing-account#create_a_new_billing_account)
        - Billing account is required for APIs used in this project
        - You will not exceed the $300 free trial setting up this project but make sure to delete the project if you do not want to be charged
        - Take note of project name created because this billing account will be used with the new project <br><br>
+ Install and initialize Google Cloud SDK by following instructions found here: [Cloud SDK Quickstart](https://cloud.google.com/sdk/docs/quickstart) <br><br>

## Step 1 - Check Prequisites Successfully Completed
Check that you have successfully installed and enabled Cloud SDK by running the config list command. If you get an error please refer to Troubleshooting steps found here [Cloud SDK Quickstart](https://cloud.google.com/sdk/docs/quickstart).  
You should see an output that includes your account along with any other configuration setup when using gcloud init

In [1]:
!gcloud config list

[accessibility]
screen_reader = False
[compute]
region = us-central1
zone = us-central1-c
[core]
account = cwilbar@alumni.nd.edu
disable_usage_reporting = False
project = spark-container-testing



Your active configuration is: [default]


In [35]:
#!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=h2qCORTEgmAkwH9QtYs2vrMNPTy77M&access_type=offline&code_challenge=KGWWdOdNXOO2Te0W4SZEOwTMxJxpOU3-V04z9QDABpc&code_challenge_method=S256


You are now logged in as [cwilbar@alumni.nd.edu].
Your current project is [spark-container-testing].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


## Step 2 - Create GCP Project

In [2]:
###### TO DO: Enter name for new project
###### Note: Proect name must be unique across GCP. If you get error when creating project please change the project name here and try again.

new_project_id = 'spark-container-testing-2'

In [3]:
!gcloud projects create {new_project_id}

Create in progress for [https://cloudresourcemanager.googleapis.com/v1/projects/spark-container-testing-2].
Waiting for [operations/cp.7465891856551616272] to finish...
..done.
Enabling service [cloudapis.googleapis.com] on project [spark-container-testing-2]...
Operation "operations/acf.p2-448485601169-a932ca8e-b263-4369-89c2-02afc6c216da" finished successfully.


In [6]:
!gcloud config set project {new_project_id}

Updated property [core/project].


#### IMPORTANT
*****TO DO: Navigate to [Cloud Console](https://console.cloud.google.com/), Change to new project, and enable billing following instructions found here: [Enable Billing](https://cloud.google.com/billing/docs/how-to/modify-project#enable_billing_for_a_project)***


## Step 3 - Enable Necessary Cloud Services

This project uses:
+ Google Kubernetes Engine for a kubernetes cluster manager
+ Google Container Registry to store spark Docker container images
  
List below contains all services needed at time of creation of this workbook. Please add/remove from this list if the names/necessary services have changed.

In [9]:
enable_services_list = [
    'bigquery.googleapis.com',
    'bigquerystorage.googleapis.com',
    'cloudapis.googleapis.com',
    'cloudbuild.googleapis.com',
    'clouddebugger.googleapis.com',
    'cloudtrace.googleapis.com',
    'compute.googleapis.com',
    'container.googleapis.com',
    'containeranalysis.googleapis.com',
    'containerregistry.googleapis.com',
    'iam.googleapis.com ',
    'iamcredentials.googleapis.com ',
    'language.googleapis.com',
    'oslogin.googleapis.com',
    'servicemanagement.googleapis.com',
    'serviceusage.googleapis.com',
    'sql-component.googleapis.com',
    'storage-api.googleapis.com',
    'storage-component.googleapis.com',
    'storage.googleapis.com'    
]

In [10]:
## Services can only be enabled 20 at a time at the time of workbook creation. Use this loop to enable 20 at a time.
for x in range(0,len(enable_services_list),20):
    !gcloud services enable {' '.join(enable_services_list[x:(x+20)])} --project={new_project_id}   

Operation "operations/acf.p2-448485601169-cc5b0618-1ffb-41b9-b130-a52f58d738ef" finished successfully.


In [12]:
# Check that services were enabled
!gcloud services list --project={new_project_id}

NAME                              TITLE
bigquery.googleapis.com           BigQuery API
bigquerystorage.googleapis.com    BigQuery Storage API
cloudapis.googleapis.com          Google Cloud APIs
cloudbuild.googleapis.com         Cloud Build API
clouddebugger.googleapis.com      Cloud Debugger API
cloudtrace.googleapis.com         Cloud Trace API
compute.googleapis.com            Compute Engine API
container.googleapis.com          Kubernetes Engine API
containeranalysis.googleapis.com  Container Analysis API
containerregistry.googleapis.com  Container Registry API
datastore.googleapis.com          Cloud Datastore API
iam.googleapis.com                Identity and Access Management (IAM) API
iamcredentials.googleapis.com     IAM Service Account Credentials API
language.googleapis.com           Cloud Natural Language API
logging.googleapis.com            Cloud Logging API
monitoring.googleapis.com         Cloud Monitoring API
oslogin.googleapis.com            Cloud OS Login API
pubsub.goo

## Step 4 - Create Necessary Service Accounts

There are two primary service accounts used in this project:  
- **Deployment Service Account**
    - We create this and add necessary roles below using the Cloud SDK
    - deployer-sa@your_project_name.iam.gserviceaccount.com
    - This account is used to deploy and test docker container and kubernetes cluster<br><br>
- **BigQuery Service Account**
    - We create this and add necessary roles below using the Cloud SDK
    - bigquery-sa@your_project_name.iam.gserviceaccount.com
    - This account is used in the container for access to big query

Check what service ccounts are already created (should be the two default ones described above)

In [13]:
!gcloud iam service-accounts list --project={new_project_id}

DISPLAY NAME                            EMAIL                                               DISABLED
Compute Engine default service account  448485601169-compute@developer.gserviceaccount.com  False


In [14]:
!gcloud iam service-accounts create deployer-sa \
    --display-name="Deployment Service Account" \
    --description="Account used to deploy to Google Cloud Project" \
    --project={new_project_id}

Created service account [deployer-sa].


In [15]:
!gcloud iam service-accounts create bigquery-sa \
    --display-name="BigQuery Service Account" \
    --description="Account used by Spark Containers to Connect to BigQuery" \
    --project={new_project_id}

Created service account [bigquery-sa].


Check service accounts were created successfully

In [16]:
!gcloud iam service-accounts list --project={new_project_id}

DISPLAY NAME                            EMAIL                                                          DISABLED
Deployment Service Account              deployer-sa@spark-container-testing-2.iam.gserviceaccount.com  False
BigQuery Service Account                bigquery-sa@spark-container-testing-2.iam.gserviceaccount.com  False
Compute Engine default service account  448485601169-compute@developer.gserviceaccount.com             False


Programatically update the roles for the new service accounts using the guide found here: [Programatic Change Access](https://cloud.google.com/iam/docs/granting-changing-revoking-access#programmatic)

In [17]:
# Save policy file in directory above where the repo is saved so that it is not stored to github
file_directory = '..\..\policy.json'

In [18]:
# Write current policy to file directory
!gcloud projects get-iam-policy {new_project_id} --format json > {file_directory}

**If running jupyter notebook run below cell to load and modify policy file.**

In [19]:
import json

with open('..\..\policy.json') as f:
    policy = json.load(f)

def modify_policy_add_role(policy, role, member):
    """Adds a new role binding to a policy."""

    binding = {"members": [member],"role": role }
    policy["bindings"].append(binding)
    return policy

members = [f'serviceAccount:deployer-sa@{new_project_id}.iam.gserviceaccount.com', 
           f'serviceAccount:bigquery-sa@{new_project_id}.iam.gserviceaccount.com']
roles = {
        members[0]:['roles/editor'],
        members[1]:['roles/bigquery.dataEditor','roles/run.serviceAgent', 'roles/bigquery.user',
                    'roles/storage.admin']}

for member in members:
    for role in roles[member]:
        policy = modify_policy_add_role(policy, role, member)

with open('..\..\policy.json', 'w') as json_file:
    json.dump(policy, json_file)

In [20]:
!gcloud projects set-iam-policy {new_project_id} {file_directory}

bindings:

Updated IAM policy for project [spark-container-testing-2].



- members:
  - serviceAccount:bigquery-sa@spark-container-testing-2.iam.gserviceaccount.com
  role: roles/bigquery.dataEditor
- members:
  - serviceAccount:bigquery-sa@spark-container-testing-2.iam.gserviceaccount.com
  role: roles/bigquery.user
- members:
  - serviceAccount:448485601169@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:service-448485601169@gcp-sa-cloudbuild.iam.gserviceaccount.com
  role: roles/cloudbuild.serviceAgent
- members:
  - serviceAccount:service-448485601169@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:service-448485601169@container-engine-robot.iam.gserviceaccount.com
  role: roles/container.serviceAgent
- members:
  - serviceAccount:service-448485601169@container-analysis.iam.gserviceaccount.com
  role: roles/containeranalysis.ServiceAgent
- members:
  - serviceAccount:service-448485601169@containerregistry.iam.gserviceaccount.com
  role: roles/co

In [21]:
# Remove policy file 
!del {file_directory}

## Step 5 - Create Kubernetes Engine Cluster

In order to deploy a container to kubernetes to run an application you first need to create a kubernetes engine cluster

In [22]:
## TO DO: Change region  to your default region
COMPUTE_REGION = 'us-central1'
CLUSTER_NAME = 'spark-cluster'
COMPUTE_ZONE = 'us-central1-c'

In [23]:
#!gcloud compute regions list

In [24]:
!gcloud config set compute/region {COMPUTE_REGION}

Updated property [compute/region].


In [25]:
!gcloud config set compute/zone {COMPUTE_ZONE}

Updated property [compute/zone].


In [26]:
# Create cluster using auto-pilot mode. This may take serveral minutes
!gcloud container clusters create {CLUSTER_NAME} \
    --project={new_project_id}

NAME           LOCATION       MASTER_VERSION   MASTER_IP       MACHINE_TYPE  NODE_VERSION     NUM_NODES  STATUS
spark-cluster  us-central1-c  1.19.9-gke.1400  104.198.69.202  e2-medium     1.19.9-gke.1400  3          RUNNING


Creating cluster spark-cluster in us-central1-c...
..................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................done.
Created [https://container.googleapis.com/v1/projects/spark-container-testing-2/zones/us-central1-c/clusters/spark-cluster].
To inspect the contents of your cluster, go to: https://console.cloud.google.com/kubernetes/workload_/gcloud/us-central1-c/spark-cluster?project=spark-container-testing-2
kubec

In [27]:
# Get credentials to use when deploying to cluster
!gcloud container clusters get-credentials {CLUSTER_NAME}

Fetching cluster endpoint and auth data.
kubeconfig entry generated for spark-cluster.


In [28]:
account = f'bigquery-sa@{new_project_id}.iam.gserviceaccount.com' 

In [29]:
# Download bigquery service account json file
!gcloud iam service-accounts keys create sa.json \
    --iam-account={account}

created key [f00f27c27428f8e7d0ff4f477de6d8d3117c4f44] of type [json] as [sa.json] for [bigquery-sa@spark-container-testing-2.iam.gserviceaccount.com]


In [30]:
# Create Kubernetes Secret from file
!kubectl create secret generic bigquery-credentials \
  --from-file ./sa.json

secret/bigquery-credentials created


## Step 6 - Create BigQuery Dataset

Your new project will need a dataset to store the data if you plan on copying/creating your own repository of data.  

This has to be a unique name per project.  

In my workflows I have named the dataset 'nba' but feel free to change it. Note that if you do change it, then you will also need to change the dataset name in any of the other python scripts in this project appropriately. 

In [31]:
dataset_name = 'amazon_reviews'

In [32]:
#Stop and re-run if this takes more than a minute
!bq --location=US mk --dataset \
--description "Stores transformed amazon review data orginally found at https://nijianmo.github.io/amazon/index.html" \
{new_project_id}:{dataset_name}  

Dataset 'spark-container-testing-2:amazon_reviews' successfully created.


## Step 7 - Build and Push Container to GCR



In [33]:
!docker build -t client-mode-spark-notebook ../client-mode

^C


#1 [internal] load build definition from Dockerfile
#1 sha256:f114d320a0272f19261b14dccc1e29a48d372e6202490fa5ecacb843176f1b4b
#1 transferring dockerfile: 3.92kB done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:5d04353d56298a3d137e7193b4accc66a64c2551e1032ecd5b771de87cafe346
#2 transferring context: 35B done
#2 DONE 0.0s

#4 [ 1/16] FROM docker.io/library/ubuntu:latest
#4 sha256:0a5f349eacf4edfd2fc1577c637ef52a2ed3280d9d5c0ab7f2e4c4052e7d6c9f
#4 DONE 0.0s

#11 https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-2.2.0.jar
#11 sha256:173f6ef3aef50bc006d358109ed87a9ceb38ec7b8570fecb583413853f5055c7
#11 DONE 0.2s

#14 https://storage.googleapis.com/spark-lib/bigquery/spark-bigquery-with-dependencies_2.12-0.20.0.jar
#14 sha256:89b5ffef7e782b56dfd9b6af19658592f873337a5886ddeb5a48695eb67d8fbf
#14 DONE 0.2s

#8 [ 5/16] RUN echo "alias pyspark=/opt/spark/bin/pyspark" >> ~/.bashrc &&     echo "alias spark-shell=/opt/spark/bin/spark-shell" >> ~/.bashrc
#8 sha256:3ea53

In [34]:
!docker tag client-mode-spark-notebook:latest gcr.io/{new_project_id}/client-mode-spark-notebook:latest

In [ ]:
!docker push gcr.io/{new_project_id}/client-mode-spark-notebook:latest

## Step 8 - Deploy App to Cluster and Expose


In [35]:
APP_NAME = 'spark-server'

In [36]:
!kubectl create deployment {APP_NAME} --image=gcr.io/{new_project_id}/client-mode-spark-notebook:latest

deployment.apps/spark-server created


In [39]:
!kubectl apply -f deployment.yaml

error: error validating "deployment.yaml": error validating data: [ValidationError(Deployment.spec.template.spec): unknown field "volumeMounts" in io.k8s.api.core.v1.PodSpec, ValidationError(Deployment.spec): unknown field "volumes" in io.k8s.api.apps.v1.DeploymentSpec]; if you choose to ignore these errors, turn validation off with --validate=false


In [37]:
!kubectl expose deployment {APP_NAME} --type LoadBalancer --port 80 --target-port 8888

service/spark-server exposed


## Step 9 - Create BigQuery View

In order to use the nba_model_game_refresh function we need to create a Big Query view that identifies what games have been loaded in to the raw_basektballrefernce_game table but have not been loaded in to the model_game_data table yet. Copying datasets does not copy views so we will always need to run this step even if you copied the entire dataset directly.

**IMPORTANT** If you ever change the number of games to use for the weighted moving average (W) then you will need to update this view as well. The game_number < filter needs to change to however many games you are averaging over. Future release will seek to remove this change dependency as it is too easy to miss.

In [ ]:
## Change dataset name (nba) if you chose a different dataset name earlier
view_name = 'nba.games_to_load_to_model'
view_query = f'CREATE OR REPLACE VIEW `{view_name}` AS \
WITH model_load_games as (SELECT \
distinct game_key as game_key \
FROM `nba.model_game` \
) \
    SELECT distinct order_of_games_per_team.game_key, \
    CASE WHEN model_load_games.game_key is NULL THEN 1 ELSE 0 END as NEEDS_TO_LOAD_TO_MODEL \
    FROM ( \
            SELECT team, game_key, row_number() OVER (PARTITION BY team ORDER BY game_date desc) as game_number \
            FROM ( \
                    SELECT \
                        home_team_name as team, game_date, game_key \
                    FROM  `nba.raw_basketballreference_game` \
                    UNION DISTINCT \
                    SELECT \
                        visitor_team_name as team, game_date, game_key \
                    FROM  `nba.raw_basketballreference_game` \
                 ) games_per_team \
            )order_of_games_per_team \
    LEFT JOIN model_load_games ON model_load_games.game_key = order_of_games_per_team.game_key \
    WHERE team in ( \
                    SELECT \
                        distinct home_team_name as team_to_load \
                    FROM `nba.raw_basketballreference_game` \
                    WHERE \
                        game_key not in (SELECT game_key FROM model_load_games) \
                    UNION DISTINCT \
                    SELECT \
                        distinct visitor_team_name as team_to_load \
                    FROM `nba.raw_basketballreference_game` \
                    WHERE \
                        game_key not in (SELECT game_key FROM model_load_games))'

run_view = f'''bq query --use_legacy_sql=false --project_id={new_project_id} "{view_query}"'''
!{run_view}

## Step 10 - Create Cloud Scheduler Jobs

This is only required if you wish to keep your data up to date. If you do not need to keep the data up to date, simply make sure you execute the nba_model_game_refresh and nba_get_upcoming_games functions once in order for the Web App to be able to function with most recent game and upcoming schedule information.

In [ ]:
## TO DO: Replace region with the region your cloud functions are deployed to and timezone with your desired scheduled time zone
region = 'us-central1'
timezone = 'America/Chicago'

In [ ]:
# Create daily scraper schedule
uri = f'https://{region}-{new_project_id}.cloudfunctions.net/nba_basketball_reference_scraper'
!gcloud scheduler jobs create http nba_basketball_reference_scraper_daily --project {new_project_id} \
--schedule "0 6 * * *" --uri {uri} --http-method GET \
--time-zone={timezone} \
--description="Calls http cloud function nba_basketball_reference_scraper every day to scrape the most recent days information and add to big query tables"

# Create daily model refresh schedule
uri = f'https://{region}-{new_project_id}.cloudfunctions.net/nba_model_game_refresh'
!gcloud scheduler jobs create http nba_model_game_refresh_daily --project {new_project_id} \
--schedule "0 7 * * *" --uri {uri} --http-method GET \
--time-zone={timezone} \
--description="Calls http cloud function nba_model_game_refresh every day to load the most recently scraped data in to the model table and most recent data for each team to firestore"

# Create upcoming games refresh schedule
uri = f'https://{region}-{new_project_id}.cloudfunctions.net/nba_get_upcoming_games'
!gcloud scheduler jobs create http nba_get_upcoming_games --project {new_project_id} \
--schedule "0 5 * * *" --uri {uri} --http-method GET \
--time-zone={timezone} \
--description="Calls http cloud function nba_get_upcoming_games every day to scrape the schedule for the upcoming week and store to cloud storage"

## Step 11 - Trigger Cloud Functions

In order to populate Firestore with the most recent game data and cloud storage with the upcoming games the fdeploy functions must be triggered. This can be done in the Console or by using the script below.

In [ ]:
empty_data = '{}'
empty_data = json.dumps(empty_data)

In [ ]:
!gcloud functions call --project {new_project_id} nba_basketball_reference_scraper --data {empty_data}

In [ ]:
!gcloud functions call nba_model_game_refresh --project {new_project_id} --data {empty_data}

In [ ]:
!gcloud functions call nba_get_upcoming_games --project {new_project_id} --data {empty_data}

## Step 12 - Create Static Model Training Data View

For tranparency and auditability we create a view using the model_game table for specific dates and a timestamped name. This will allow us to come back to train different models on the same data. These are created as views so they are not part of what is copied externally but you could create these as tables instead if desired but would have to pay for additional storage costs.

In [ ]:
#TO DO: Change timezone to your timezone if desired
timezone = 'America/Chicago'

In [ ]:
## Create view that excludes first game in every seasons because rest days will be way off. 
#It will use moving average dating back to previous season.

query = f"""EXECUTE IMMEDIATE CONCAT(' \
                CREATE OR REPLACE VIEW `nba.model_training_data_' \
                , FORMAT_DATE('%Y%m%d', CURRENT_DATE(\\"{timezone}\\")) \
            ,'` AS \
                SELECT * FROM ( \
                    SELECT \
                        *, \
                        ROW_NUMBER() OVER (PARTITION BY g.SEASON, g.TEAM ORDER BY g.game_date asc) as SEASON_GAME_NUMBER, \
                    FROM nba.model_game g \
            ) WHERE SEASON_GAME_NUMBER > 1 and is_home_team = 1 \
                and game_date < DATE_SUB(CURRENT_DATE(\\"{timezone}\\"), INTERVAL 1 WEEK)')"""

run_query = f'''bq query --use_legacy_sql=false --project_id={new_project_id} "{query}"'''

!{run_query}

## Step 13 - Create Baseline Linear Model using View

We will now use the data in the view we just created to generate a linear model on all of the relevant variables. 

You definiltey will want to open the Console to explore the model further but that is left as a separate task.

**NOTE:** This is the most time consuming and costly step. Be careful with running this too many times but definitely expirement with different modeling types.

In [ ]:
## If running Step 13 on the same date as Step 12 execute this cell to set the view date
from datetime import datetime
view_date = datetime.now().strftime('%Y%m%d')

##If running Step 13 on a different day than Step 12 change the date here to the date you created the view in Step 12  
view_date = '20210310'

In [ ]:
W = 20

In [ ]:
model_query = f"""CREATE OR REPLACE MODEL nba.baseline_linear_model \
  OPTIONS(model_type='LINEAR_REG', input_label_cols=['spread']) \
    AS SELECT spread, \
        is_home_team, \
        incoming_is_win_streak, \
        incoming_is_win_streak_opponent, \
        incoming_wma_{W}_pace, \
        incoming_wma_{W}_efg_pct, \
        incoming_wma_{W}_tov_pct, \
        incoming_wma_{W}_ft_rate, \
        incoming_wma_{W}_off_rtg, \
        incoming_wma_{W}_opponent_efg_pct, \
        incoming_wma_{W}_opponent_tov_pct, \
        incoming_wma_{W}_opponent_ft_rate, \
        incoming_wma_{W}_opponent_off_rtg, \
        incoming_wma_{W}_starter_minutes_played_proportion, \
        incoming_wma_{W}_bench_plus_minus,\
        incoming_wma_{W}_opponnent_starter_minutes_played_proportion, \
        incoming_wma_{W}_opponent_bench_plus_minus, \
        incoming_rest_days - incoming_rest_days_opponent as rest_days_difference \
    FROM `nba.model_training_data_{view_date}`"""

model_query = f'''bq query --use_legacy_sql=false --project_id={new_project_id} "{model_query}"'''

!{model_query}

## Step 14 Deploy App Engine App

We are finally ready to deploy the app engine web app! If you have sucessfully completed all steps above then you should be able to navigate to a webpage that works the same as the [webpage](https://nba-predictions-prod.uc.r.appspot.com/) in the Readme.

As a prequisite, make sure you are running this notebook in the folder from the gitclone or be sure to replace the file paths below with the correct file path. 

In [ ]:
!gcloud app deploy ../webapp/app.yaml --project={new_project_id} --promote --quiet
print(f'Check you your new web page at https://{new_project_id}.uc.r.appspot.com/')

## Optional - Delete Project

To avoid on-going charges for everything created in this workbook run the below command to delete the project that you just created. Note it will take approximately 30 days for full completion and you will stil be charged for any charges accrued during this walkthrough. Check out [Deleting GCP Project](https://cloud.google.com/resource-manager/docs/creating-managing-projects?visit_id=637510410447506984-2569255859&rd=1#shutting_down_projects) for more information.

In [ ]:
### Uncomment code to delete project
# !gcloud projects delete {new_project_id}